### About this dataset:



Cardiovascular diseases (CVDs) are the number 1 cause of death globally, taking an estimated 17.9 million lives each year, which accounts for 31% of all deaths worlwide. Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

Most cardiovascular diseases can be prevented by addressing behavioural risk factors such as tobacco use, unhealthy diet and obesity, physical inactivity and harmful use of alcohol using population-wide strategies.

People with cardiovascular disease or who are at high cardiovascular risk (due to the presence of one or more risk factors such as hypertension, diabetes, hyperlipidaemia or already established disease) need early detection and management wherein a machine learning model can be of great help.

### Task:
    
Create a model to assess the likelihood of a death by heart failure event.

### Steps Involved:
1. [Reading and understanding the data](#1)
2. [Exploratory Data Analysis (EDA)](#2)
3. [Performing Train-Test Split](#3)
4. [Handling data imbalance using SMOTE](#4)
5. [Feature Engineering](#5)
    - [Feature Selection](#5.1)
    - [Feature Scaling](#5.2)
6. [Model Building/Evaluation](#6)

<a id="1"></a>
# 1. Reading and understanding the data

In [ ]:
# importing prerequisites
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

%matplotlib inline
pd.set_option("display.max_rows", None,"display.max_columns", None)
warnings.simplefilter(action='ignore')
plt.style.use('seaborn')

In [ ]:
# Importing dataset
df = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Checking numerical attributes stats, to identify if these is any abnormality
df.describe()

In [ ]:
df.shape

In [ ]:
# Checking if there are any missing values
df.isnull().sum()

Since everything seems normal in the dataset, we can skip Data Formatting step and move directly to EDA.

<a id="2"></a>
# 2. Exploratory Data Analysis (EDA)

## a) Univariate Analysis

**Categorical Columns**

In [ ]:
# Visualizing categorical features
cat_features = ["anaemia","diabetes","high_blood_pressure","sex","smoking","DEATH_EVENT"]

i=0
while i < 6:
    fig = plt.figure(figsize=[8,3]) 

    plt.subplot(1,2,1)
    sns.countplot(x=cat_features[i], data=df)
    i += 1
    
    plt.subplot(1,2,2)
    sns.countplot(x=cat_features[i], data=df)
    i += 1
    
    plt.show()

**Numerical Columns**

In [ ]:
# Visualizing numerical columns

# Extracting numerical columns using categorical columns
num_features = df.columns[~df.columns.isin(cat_features)].tolist()
r = c = 0
fig,ax = plt.subplots(4,2,figsize=(15,14))

for n,i in enumerate(num_features):
    sns.boxplot(x=i, data=df,ax=ax[r,c])
    c+=1
    if (n+1)%2==0:
        r+=1
        c=0
ax[r,c].axis("off")
plt.show()

As can be observed from above boxplots 'creatinine_phosphokinase', 'platelets' & 'serum_creatinine' are having outliers which might affect the decision making of our model. There are various techniques using which one can treat outliers like capping the values or by removing the entire row. But for taking such decisions, one must have domain knowledge. So for now we will be keeping the them as it is.  

## b) Bivariate Analysis

**Categorical features vs target variable**

In [ ]:
cat_features = ["anaemia","diabetes","high_blood_pressure","sex","smoking","DEATH_EVENT"]
r = c = 0
fig,ax = plt.subplots(3,2,figsize=(10,12))

for n,i in enumerate(cat_features[:-1]):
    temp = df.pivot_table(columns=df.DEATH_EVENT,index=df[i],aggfunc='count')['age']
    temp.plot.bar(stacked=True,ax=ax[r,c])
    ax[r,c].set_ylabel("count of observations")
    c+=1
    if (n+1)%2==0:
        r+=1
        c=0
ax[r,c].axis("off")
plt.show()

- It seems that "male" patients have higher tendency get heart problems.
- Above distribution suggests that smoking is not the root cause behind Cardiovascular diseases(CVDs).
- There are fewer cases of CVDs patients that have high blood pressure. 

**Numerical features vs target variable**

In [ ]:
# Extracting numerical columns using categorical columns
num_features = df.columns[~df.columns.isin(cat_features)].tolist()
r = c = 0
fig,ax = plt.subplots(4,2,figsize=(10,20))

for n,i in enumerate(num_features):
    sns.boxplot(x='DEATH_EVENT',y=i, data=df,ax=ax[r,c])
    c+=1
    if (n+1)%2==0:
        r+=1
        c=0
ax[r,c].axis("off")
plt.show()

- Generally patients who die due to CVDs are older then the patients who survive.
- Concentation of 'serum_sodium' and 'ejection_fraction' are higher for patients who survive CVDs.
- 'serum_creatinine' is lower for patients who survive CVDs.

<a id="3"></a>
# 3. Performing Train-Test split

In [ ]:
from sklearn.model_selection import train_test_split

y = df['DEATH_EVENT'] 
X = df.drop('DEATH_EVENT',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

print(f'X_train order : {X_train.shape}')
print(f'y_train order : {y_train.shape}')
print(f'X_test order : {X_test.shape}')
print(f'y_test order :{y_test.shape}')

<a id="4"></a>
# 4. Handling data imbalance using SMOTE

Imbalanced classification involves developing predictive models on classification datasets that have a severe class imbalance (eg. ratio of 90:10). These kind of senarios are usually common in Healthcare and BSFI domain. eg: In Credit risk analysis, where the credit worthiness of a potential borrowers is determined, chances of approving a bad loan are very low thus making the target variable very skewed.

The challenge of working with imbalanced datasets is that most machine learning techniques will ignore, and in turn have poor performance on, the minority class, although typically it is performance on the minority class that is most important.

One approach to addressing imbalanced datasets is to oversample the minority class. The simplest approach involves duplicating examples in the minority class, although these examples don’t add any new information to the model. Instead, new examples can be synthesized from the existing examples. This is a type of data augmentation for the minority class and is referred to as the Synthetic Minority Oversampling Technique, or SMOTE for short. <a href="https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/">READ MORE...</a>

In [ ]:
# Target distribution
plt.figure(figsize=(3.5,3))
sns.countplot(df['DEATH_EVENT'])
plt.title("Target Class Distribution")
plt.show()

Above plot shows that our dataset is moderately imbalanced. So, performing SMOTE analysis will not make any sense. Moreover since our dataset has very few training samples, performing SMOTE will reduce our model accuracy drastically.

But for the sake of understanding lets see what are the steps involved in SMOTE analysis.

**Creating Synthetic samples using SMOTE Analysis**

In [ ]:
from imblearn.over_sampling import SMOTE

smote_up = SMOTE(random_state = 42)
X_train_res , y_train_res = smote_up.fit_sample(X_train,y_train)

In [ ]:
from collections import Counter
# Target distribution post SMOTE Analysis
print("Class distribution without SMOTE:",Counter(y_train))
print("Class distribution with SMOTE:",Counter(y_train_res))

We can see that count of samples in class '1' have increased from '82' to '157' making the ratio of target classes 1:1.

<a id="5"></a>
# 5. Feature Engineering

<a id="5.1"></a>
## a) Feature Selection

We will be using RFE which is a wrapper based method for selecting best features.

In [ ]:
from sklearn.feature_selection import RFE
def top_feat(model,feat_cnt):
    '''
    This function provides the top "n" features for provided model using RFE.
    '''
    rfe = RFE(model, n_features_to_select=feat_cnt)
    rfe.fit(X_train, y_train)
    return X.columns[rfe.support_].to_list()

***There are other types of feature selection techniques which are mentioned below.***

#### i) Filter Based Methods

Features are selected on the basis of their scores in various statistical tests for their correlation with the outcome variable. Filter methods are much faster compared to wrapper methods as they do not involve training the models. We need to take care of multicollinearity beforehand. Eg:
- Pearson Correlation
- Chi-Square test
- ANOVA
- LDA

<img src="https://www.analyticsvidhya.com/wp-content/uploads/2016/11/FS1.png"></img>

In [ ]:
# Saving 'DEATH_EVENT' corr. to a DataFrame
Feature_Imp = df.corr()['DEATH_EVENT'].reset_index()
Feature_Imp.drop(12,inplace=True)

In [ ]:
# Converting Pearsons corr. to its absolute value
Feature_Imp.rename(columns={'DEATH_EVENT':'Peason_Corr'},inplace = True)
Feature_Imp['Peason_Corr'] = Feature_Imp['Peason_Corr'].abs()

In [ ]:
Feature_Imp.sort_values(by='Peason_Corr',ascending=False,ignore_index=True,inplace = True)
Feature_Imp

#### ii) Wrapper Based Method

Use a subset of features and train a model using them. Based on the inferences that we draw from the previous model, we decide to add or remove features from your subset. These methods are usually computationally very expensive. One example of this is RFE i.e. Recursive Feature Elimination which uses"coef_" or "feature_importances_" to select best features.

- Recusive Feature Elimination (RFE)
- Backward Elimination
- Forward Selection

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [ ]:
# Selecting top 6 features using RFE for Logestic Regression Model
rfe = RFE(estimator=LogisticRegression(), n_features_to_select=6, verbose=1)
rfe.fit(X_train, y_train)
rfe_df = pd.DataFrame(list(zip(X.columns, rfe.support_, rfe.ranking_)),columns= ['index','RFE_Support','RFE_rank'])
rfe_df

In [ ]:
# Columns to be excluded for building Logestic Regression Model
X.columns[~rfe.support_].to_list()

In [ ]:
# Columns to be included for building Logestic Regression Model
X.columns[rfe.support_].to_list()

#### iii) Embedded Methods

Embedded methods combine the qualities of filter and wrapper methods. It’s implemented by algorithms that have their own built-in feature selection methods. 
eg:
- Lasso regression performs L1 regularization
- Ridge regression performs L2 regularization
- feature_importances_ in Decision tree based models

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=500)
rf.fit(X_train,y_train)

In [ ]:
df_feat_imp = pd.DataFrame(list(zip(X_train.columns,rf.feature_importances_)),columns = ['index','RF_fe'])
df_feat_imp.sort_values(by='RF_fe', ascending=False, ignore_index=True, inplace=True)
df_feat_imp.plot.barh(x='index',y='RF_fe')

<a id="5.2"></a>
## b) Feature Scaling

##### When to Scale ?
Rule of thumb I follow here is any algorithm that computes distance or assumes normality, scale your features!!!

Some examples of algorithms where feature scaling matters are:

- k-nearest neighbors with an Euclidean distance measure is sensitive to magnitudes and hence should be scaled for all features to weigh in equally.
- Scaling is critical, while performing Principal Component Analysis(PCA). PCA tries to get the features with maximum variance and the variance is high for high magnitude features. This skews the PCA towards high magnitude features.
- We can speed up gradient descent by scaling. This is because θ will descend quickly on small ranges and slowly on large ranges, and so will oscillate inefficiently down to the optimum when the variables are very uneven.
- Tree based models are not distance based models and can handle varying ranges of features. Hence, Scaling is not required while modelling trees.
- Algorithms like Linear Discriminant Analysis(LDA), Naive Bayes are by design equipped to handle this and gives weights to the features accordingly. Performing a features scaling in these algorithms may not have much effect.

In [ ]:
# Performing Standard Scaler on X train and test
from sklearn.preprocessing import StandardScaler

def standard_scaler():
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train),columns=X.columns)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test),columns=X.columns)
    return X_train_scaled,X_test_scaled

## 6. Model Creation/Evaluation

1. Logestic Regression
2. K Nearest Neighbour (KNN)
3. Naive Bayers
4. Support Vector machine (SVM)
5. Decision Tree
6. Ensembled Techniques
    - Random Forest
    - Gradient Boosting (GBM)
    - XGboost
    - CatBoost
    - LightGBM

In [ ]:
from sklearn.metrics import accuracy_score, plot_confusion_matrix, classification_report, plot_roc_curve

Acc_list = []

def hf_model(model,model_name,features=""):
    # Feature Scaling
    if model_name in ['GaussianNB','XGboost','RandomForest','GradientBoosting',
                      'LGBMClassifier','CatBoostClassifier','DecisionTree']:
        X_train_temp,X_test_temp = X_train,X_test
    else:
        X_train_temp,X_test_temp = standard_scaler()
    
    # Feature Selection
    if model_name not in ['KNN','GaussianNB','SVM']:
        X_train_temp = X_train_temp[features]
        X_test_temp = X_test_temp[features]
   
    # Fitting train set
    model.fit(X_train_temp,y_train)
    
    # Performing prediction on test set
    y_test_pred = model.predict(X_test_temp)
        
    # Calculating Evaluation Metrics
    s1 = accuracy_score(y_test,y_test_pred)
    print(f"{model_name} Success Rate :{round(s1*100,2)}%\n")
    print(classification_report(y_test,y_test_pred))

    
    # Plotting Confusion Matrix and ROC curve
    fig, ax = plt.subplots(1,2,figsize=(12,5))
    plot_confusion_matrix(model,X_test_temp,y_test,cmap='Blues',ax=ax[0])
    ax[0].set_title("Confusion Matrix",fontsize=16)
    ax[0].set_xticks([0,1], ["Heart Not Failed","Heart Fail"])
    ax[0].set_yticks([0,1], ["Heart Not Failed","Heart Fail"])
    ax[0].grid(False)
    
    plot_roc_curve(model,X_test_temp,y_test,ax=ax[1])
    ax[1].set_title("ROC Curve",fontsize=16)
    plt.show()

### a) Logestic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(n_jobs=-1)
features = top_feat(lr,10)
hf_model(lr,"Logestic Regression",features)

### b) K Nearest Neighbour (KNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7)
hf_model(knn,"KNN")

### c) Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
hf_model(gnb,"GaussianNB")

### d) Support Vector machine (SVM)

In [ ]:
from sklearn.svm import SVC

sv = SVC()
hf_model(sv,"SVM")

### e) Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV

dt = DecisionTreeClassifier()

# criteria for splitting nodes
criterion = ["gini", "entropy"]
# Maximum number of levels in a tree
max_depth = list(range(4,17,4))
# The minimum number of samples required to be at a leaf node
min_samples_leaf=[1,2,5,7]
# Minimum number of samples required to split an internal node
min_samples_split=list(range(2,9,2))

# Hyperparameters dict
param_grid = {"max_depth":max_depth,
              "min_samples_split":min_samples_split,
              "min_samples_leaf":min_samples_leaf,
              "criterion":criterion}

# Performing RFE
features = top_feat(dt,10)

# Using Stratified K-fold to train the model
skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 0)

dt_rs = RandomizedSearchCV(estimator = dt, param_distributions = param_grid,
                           cv = skf.split(X_train[features],y_train))

hf_model(dt_rs,"DecisionTree",features)

### g) Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1)

# Number of trees in Random forest
n_estimators=list(range(400,1000,100))
# Maximum number of levels in a tree
max_depth = list(range(4,13,4))
# The minimum number of samples required to be at a leaf node
min_samples_leaf=[1,2,5,7]
# Minimum number of samples required to split an internal node
min_samples_split=list(range(2,9,2))
# Number of fearures to be considered at each split
max_features=['auto','sqrt']


# Hyperparameters dict
param_grid = {"n_estimators":n_estimators,
              "max_depth":max_depth,
              "min_samples_split":min_samples_split,
              "min_samples_leaf":min_samples_leaf,
              "max_features":max_features}
# Performing RFE
features = top_feat(rf,10)

# Using Stratified K-fold to train the model
skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 0)

rf_rs = RandomizedSearchCV(estimator = rf, param_distributions = param_grid,
                           cv = skf.split(X_train[features],y_train))

hf_model(rf_rs,"RandomForest",features)

### h) Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()

# Rate at which correcting is being made
learning_rate = [0.01, 0.1, 0.2,0.3]
# Number of trees in Gradient boosting
n_estimators=list(range(400,1000,100))
# Maximum number of levels in a tree
max_depth=list(range(4,13,4))
# Minimum number of samples required to split an internal node
min_samples_split=list(range(2,9,2))
# Minimum number of samples required to be at a leaf node.
min_samples_leaf=[1,2,5,7]
# Number of fearures to be considered at each split
max_features=['auto','sqrt']

# Hyperparameters dict
param_grid = {"learning_rate":learning_rate,
              "n_estimators":n_estimators,
              "max_depth":max_depth,
              "min_samples_split":min_samples_split,
              "min_samples_leaf":min_samples_leaf,
              "max_features":max_features}

# Performing RFE
features = top_feat(gb,10)

# Using Stratified K-fold to train the model
skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 0)

gb_rs = RandomizedSearchCV(estimator = gb, param_distributions = param_grid,
                           cv = skf.split(X_train[features],y_train))

hf_model(gb_rs,"GradientBoosting",features)

In [ ]:
gb_rs.best_params_

### i) XGBoost Classifier

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_jobs=-1,learning_rate=0.01, n_estimators=1000, objective='binary:logistic')

# Hyperparameters dict
param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

# Performing RFE
features = top_feat(xgb,10)

# Using Stratified K-fold to train the model
skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 0)

xgb_rs = RandomizedSearchCV(estimator = xgb, param_distributions = param_grid,
                           cv = skf.split(X_train[features],y_train))

hf_model(xgb_rs,"XGboost",features)

### j) LGBM Classifier

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(n_jobs=-1,learning_rate=0.01, n_estimators=1000)
features = top_feat(lgb,10)
hf_model(lgb,"LGBMClassifier",features)

### k) CatBoost Classifier

In [ ]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(learning_rate=0.01, n_estimators=1000,verbose=0)
features = top_feat(cat,10)
hf_model(cat,"CatBoostClassifier",features)